In [25]:
# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.evaluation import MulticlassMetrics

# preds = df.select(['label', 'features']) \
#                             .df.map(lambda line: (line[1], line[0]))
# metrics = MulticlassMetrics(preds)

    # Confusion Matrix
# print(metrics.confusionMatrix().toArray())```

In [61]:
#import numpy
#import matplotlib.pyplot as plt

#x = numpy.random.uniform(0.0, 5.0, 250)

#plt.hist(x, 5)
#plt.show()

In [43]:
#vzorka_cela.crosstab("Accident_Severity", "Day_of_Week").show()

+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|Accident_Severity_Day_of_Week|    1|    2|    3|    4|    5|    6|    7|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|                            2|23942|29487|30915|30958|31556|34732|28784|
|                            1|  649|  542|  500|  495|  539|  741|  737|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+



In [ ]:
#vzorka_cela.saveAsTextFile('small_data.csv')

In [ ]:
#vzorka_cela.write.format('com.databricks.spark.csv').mode('append').option("header", "true").save('C:/Users/FIlip/Documents/TSVD/Zadanie')

In [1]:
import sys
from pyspark import SparkContext

sc = SparkContext(appName="example23")

print(u'Python version ' + sys.version)
print(u'Spark version ' + sc.version)

Python version 2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:53:40) [MSC v.1500 64 bit (AMD64)]
Spark version 2.2.0


In [2]:
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import sys
from pyspark import SparkContext

spark = SparkSession.builder.appName("Zadanie").getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
# nacitanie
data=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Accidents.csv')
data1=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Vehicles.csv')
data2=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Casualties.csv')

In [4]:
data1=data1.withColumnRenamed("Accident_Index","ID")
data2=data2.withColumnRenamed("Accident_Index","IDE")

In [5]:
#spojenie dat a vymazanie potom tych duplikatych ID stlpcov
merge = data.join(data1, data.Accident_Index == data1.ID)
merge.drop("ID")
full_merge = merge.join(data2, merge.Accident_Index == data2.IDE)
full_merge=full_merge.drop("IDE")
full_merge=full_merge.drop("ID")
full_merge=full_merge.drop("Vehicle_Reference")

len(full_merge.columns)
full_merge
#print(full_merge.show())

DataFrame[Accident_Index: string, Location_Easting_OSGR: int, Location_Northing_OSGR: int, Longitude: double, Latitude: double, Police_Force: int, Accident_Severity: int, Number_of_Vehicles: int, Number_of_Casualties: int, Date: string, Day_of_Week: int, Time: string, Local_Authority_(District): int, Local_Authority_(Highway): string, 1st_Road_Class: int, 1st_Road_Number: int, Road_Type: int, Speed_limit: int, Junction_Detail: int, Junction_Control: int, 2nd_Road_Class: int, 2nd_Road_Number: int, Pedestrian_Crossing-Human_Control: int, Pedestrian_Crossing-Physical_Facilities: int, Light_Conditions: int, Weather_Conditions: int, Road_Surface_Conditions: int, Special_Conditions_at_Site: int, Carriageway_Hazards: int, Urban_or_Rural_Area: int, Did_Police_Officer_Attend_Scene_of_Accident: int, LSOA_of_Accident_Location: string, Vehicle_Type: int, Towing_and_Articulation: int, Vehicle_Manoeuvre: int, Vehicle_Location-Restricted_Lane: int, Junction_Location: int, Skidding_and_Overturning: in

In [6]:
# čiže ci cieľovy atribut je Accident_Severity a ma 3 hodnoty : 1, 2, 3 - jedna je smrtelna 2 je že važna ale nezomrel a 3 je že ľahka... tak som 3 replacol na 2 aby bolo len smrtelna
# a niesmrtelna
from pyspark.sql.functions import when
newsdf = full_merge.withColumn("Accident_Severity", when(full_merge["Accident_Severity"] == 3, 2).otherwise(full_merge["Accident_Severity"]))

In [7]:
newsdf.count()

4287593

In [8]:
#cize sampling  som si vytiahol koľko je počet mrtvych a koľko počet čo prežili nehodu a podľa toho som v tokom pomere zmenšil tie data na 5%
newsdf.registerTempTable("TempTable")
mrtvy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 1')
mrtvy_pocet = mrtvy.count()
zivy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 2')
zivy_pocet = zivy.count()

vzorka_mrtvy = mrtvy.sampleBy("Accident_Severity", fractions = {1: 0.05}, seed = 0)
vzorka_zivy = zivy.sampleBy("Accident_Severity", fractions = {2: 0.05}, seed = 0)

#tu ich nazad spojim a s tym datasetom treba pracovať ďalej
vzorka_cela = vzorka_mrtvy.union(vzorka_zivy)

In [9]:
#rozdelenie na trenovaciu a testovaciu 
training_data, test_data = vzorka_cela.randomSplit([0.6, 0.4], seed=123)

In [10]:
vzorka_cela.count()

214577

In [11]:
print(vzorka_cela.take(1))
print("------")

[Row(Accident_Index=u'200522DB43935', Location_Easting_OSGR=385650, Location_Northing_OSGR=273200, Longitude=-2.212142, Latitude=52.356639, Police_Force=22, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=3, Date=u'03/04/2005', Day_of_Week=1, Time=u'15:41', Local_Authority_(District)=278, Local_Authority_(Highway)=u'E10000034', 1st_Road_Class=3, 1st_Road_Number=450, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=1, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01032431', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=17, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [12]:
print(training_data.take(1))
print("-----------")

[Row(Accident_Index=u'200522DB43935', Location_Easting_OSGR=385650, Location_Northing_OSGR=273200, Longitude=-2.212142, Latitude=52.356639, Police_Force=22, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=3, Date=u'03/04/2005', Day_of_Week=1, Time=u'15:41', Local_Authority_(District)=278, Local_Authority_(Highway)=u'E10000034', 1st_Road_Class=3, 1st_Road_Number=450, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=1, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01032431', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=17, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [13]:
print(test_data.take(1))

[Row(Accident_Index=u'200532D088205', Location_Easting_OSGR=498070, Location_Northing_OSGR=389760, Longitude=-0.526598, Latitude=53.395558, Police_Force=32, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=9, Date=u'18/12/2005', Day_of_Week=1, Time=u'18:00', Local_Authority_(District)=356, Local_Authority_(Highway)=u'E10000019', 1st_Road_Class=3, 1st_Road_Number=631, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=6, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01026411', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=16, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [14]:
# #najprv si dropnem tie atributy ktore tam nechcem mat
vzorka_cela = vzorka_cela.drop(
					 "Local_Authority_(Highway)",
 					 "LSOA_of_Accident_Location",
					 "Time",
					 "Date",
					 "Accident_Index")


In [15]:
#do premennej names si ulozim mena atributov
names = vzorka_cela.schema.names
print(names)

#print(names.count)
#vytvorim si prazdny list correlations a do neho vo for cykle hodim korelacie kazdeho atributu s #atributom accident_severity, ale dajak to nefujguje zatial :D

correlations = []
for name in names:
    correlations.extend([vzorka_cela.stat.corr('Accident_Severity',name)])

['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Day_of_Week', 'Local_Authority_(District)', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Vehicle_Type', 'Towing_and_Articulation', 'Vehicle_Manoeuvre', 'Vehicle_Location-Restricted_Lane', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_in_Carriageway', 'Vehicle_Leaving_Carriageway', 'Hit_Object_off_Carriageway', '1st_Point_of_Impact', 'Was_Vehicle_Left_Hand_Drive?', 'Journey_Purpose_of_Driver', 'Sex_of_Driver', 'Age_of_Driver', 'Age_Band_of_Driver', 'En

In [16]:
print(correlations)

[0.014786428056535635, -0.01621638844788331, 0.014881649693193258, -0.01588609737790027, -0.037619583333479395, 1.0, -0.06369110692696972, -0.12066636855785076, -0.0031986647821150064, -0.03773716887909421, 0.053598702883446515, 0.006687154872887731, -0.006109255069014147, -0.11069335025575402, 0.05187111068738224, 0.07414108021980696, 0.07442515761683721, 0.020083848899277217, 0.006897313588318905, 0.025252285899736878, -0.04905971443910548, -0.0109911636809659, -0.004272140184665999, 0.005466851521745538, -0.010512147510305117, -0.09896599141093311, 0.05657186361984607, -0.021262339309549197, -0.02388275653412297, -0.05971450692822328, -0.008074087360074816, 0.053147172978235534, -0.05192407596094765, -0.02176826954569431, -0.06959285988824362, -0.05299760866808701, 0.02486474760408355, 0.01413789292853396, -0.0034677890495716335, 0.03707975986638938, -0.03187575633656105, -0.03147575804289188, -0.040036471882442866, -0.012458325547380342, -0.006748442902452876, -0.001221763038766307

In [17]:
names = vzorka_cela.schema.names
t = zip(names,correlations)
#print(t)
#print("-------------------")
tt = spark.createDataFrame(t)
#tt.show()

tt.registerTempTable("TempTable")
atributy_table = sqlContext.sql('SELECT * FROM TempTable WHERE _2 > 0.05 OR _2 <-0.05')
atributy = atributy_table.select("_2")
atributy_table.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|   Accident_Severity|                 1.0|
|  Number_of_Vehicles|-0.06369110692696972|
|Number_of_Casualties|-0.12066636855785076|
|      1st_Road_Class|0.053598702883446515|
|         Speed_limit|-0.11069335025575402|
|     Junction_Detail| 0.05187111068738224|
|    Junction_Control| 0.07414108021980696|
|      2nd_Road_Class| 0.07442515761683721|
| Urban_or_Rural_Area|-0.09896599141093311|
|Did_Police_Office...| 0.05657186361984607|
|   Vehicle_Manoeuvre|-0.05971450692822328|
|   Junction_Location|0.053147172978235534|
|Skidding_and_Over...|-0.05192407596094765|
|Vehicle_Leaving_C...|-0.06959285988824362|
|Hit_Object_off_Ca...|-0.05299760866808701|
|  Casualty_Reference|-0.10583313411571128|
|   Casualty_Severity|   0.423926240717267|
+--------------------+--------------------+



In [18]:
vzorka_cela.count()

214577

In [19]:
Accident_Severity = vzorka_cela.describe(["Accident_Severity"])
Accident_Severity.collect()

[Row(summary=u'count', Accident_Severity=u'214577'),
 Row(summary=u'mean', Accident_Severity=u'1.9804126257707024'),
 Row(summary=u'stddev', Accident_Severity=u'0.13857777057006804'),
 Row(summary=u'min', Accident_Severity=u'1'),
 Row(summary=u'max', Accident_Severity=u'2')]

In [20]:
Number_of_Vehicles = vzorka_cela.describe(["Number_of_Vehicles"])
Number_of_Vehicles.collect()

[Row(summary=u'count', Number_of_Vehicles=u'214577'),
 Row(summary=u'mean', Number_of_Vehicles=u'2.363244895771681'),
 Row(summary=u'stddev', Number_of_Vehicles=u'2.5958718437844035'),
 Row(summary=u'min', Number_of_Vehicles=u'1'),
 Row(summary=u'max', Number_of_Vehicles=u'67')]

In [21]:
Number_of_Casualties = vzorka_cela.describe(["Number_of_Casualties"])
Number_of_Casualties.collect()

[Row(summary=u'count', Number_of_Casualties=u'214577'),
 Row(summary=u'mean', Number_of_Casualties=u'2.1372374485615886'),
 Row(summary=u'stddev', Number_of_Casualties=u'3.446195182948457'),
 Row(summary=u'min', Number_of_Casualties=u'1'),
 Row(summary=u'max', Number_of_Casualties=u'93')]

In [22]:
Road_Class = vzorka_cela.describe(["1st_Road_Class"])
Road_Class.collect()

[Row(summary=u'count', 1st_Road_Class=u'214577'),
 Row(summary=u'mean', 1st_Road_Class=u'3.9091281917446885'),
 Row(summary=u'stddev', 1st_Road_Class=u'1.4513059991649084'),
 Row(summary=u'min', 1st_Road_Class=u'1'),
 Row(summary=u'max', 1st_Road_Class=u'6')]

In [23]:
Speed_limit = vzorka_cela.describe(["Speed_limit"])
Speed_limit.collect()

[Row(summary=u'count', Speed_limit=u'214577'),
 Row(summary=u'mean', Speed_limit=u'41.07348411059899'),
 Row(summary=u'stddev', Speed_limit=u'15.123157400089896'),
 Row(summary=u'min', Speed_limit=u'10'),
 Row(summary=u'max', Speed_limit=u'70')]

In [24]:
Junction_Detail = vzorka_cela.describe(["Junction_Detail"])
Junction_Detail.collect()

[Row(summary=u'count', Junction_Detail=u'214577'),
 Row(summary=u'mean', Junction_Detail=u'2.376899667718348'),
 Row(summary=u'stddev', Junction_Detail=u'2.6154558806551593'),
 Row(summary=u'min', Junction_Detail=u'-1'),
 Row(summary=u'max', Junction_Detail=u'9')]

In [25]:
Junction_Control = vzorka_cela.describe(["Junction_Control"])
Junction_Control.collect()

[Row(summary=u'count', Junction_Control=u'214577'),
 Row(summary=u'mean', Junction_Control=u'1.7916971530033508'),
 Row(summary=u'stddev', Junction_Control=u'2.302666453659238'),
 Row(summary=u'min', Junction_Control=u'-1'),
 Row(summary=u'max', Junction_Control=u'4')]

In [26]:
nd_Road_Class = vzorka_cela.describe(["2nd_Road_Class"])
nd_Road_Class.collect()

[Row(summary=u'count', 2nd_Road_Class=u'214577'),
 Row(summary=u'mean', 2nd_Road_Class=u'2.6265536380879593'),
 Row(summary=u'stddev', 2nd_Road_Class=u'3.211564003149304'),
 Row(summary=u'min', 2nd_Road_Class=u'-1'),
 Row(summary=u'max', 2nd_Road_Class=u'6')]

In [27]:
Urban_or_Rural_Area = vzorka_cela.describe(["Urban_or_Rural_Area"])
Urban_or_Rural_Area.collect()

[Row(summary=u'count', Urban_or_Rural_Area=u'214577'),
 Row(summary=u'mean', Urban_or_Rural_Area=u'1.4045261141688066'),
 Row(summary=u'stddev', Urban_or_Rural_Area=u'0.49095314546545527'),
 Row(summary=u'min', Urban_or_Rural_Area=u'1'),
 Row(summary=u'max', Urban_or_Rural_Area=u'3')]

In [28]:
Did_Police_Officer_Attend_Scene_of_Accident = vzorka_cela.describe(["Did_Police_Officer_Attend_Scene_of_Accident"])
Did_Police_Officer_Attend_Scene_of_Accident.collect()

[Row(summary=u'count', Did_Police_Officer_Attend_Scene_of_Accident=u'214577'),
 Row(summary=u'mean', Did_Police_Officer_Attend_Scene_of_Accident=u'1.1567455971516052'),
 Row(summary=u'stddev', Did_Police_Officer_Attend_Scene_of_Accident=u'0.36963810518369733'),
 Row(summary=u'min', Did_Police_Officer_Attend_Scene_of_Accident=u'-1'),
 Row(summary=u'max', Did_Police_Officer_Attend_Scene_of_Accident=u'3')]

In [29]:
Vehicle_Manoeuvre = vzorka_cela.describe(["Vehicle_Manoeuvre"])
Vehicle_Manoeuvre.collect()

[Row(summary=u'count', Vehicle_Manoeuvre=u'214577'),
 Row(summary=u'mean', Vehicle_Manoeuvre=u'12.722766186497156'),
 Row(summary=u'stddev', Vehicle_Manoeuvre=u'6.176489972934277'),
 Row(summary=u'min', Vehicle_Manoeuvre=u'-1'),
 Row(summary=u'max', Vehicle_Manoeuvre=u'18')]

In [30]:
Junction_Location = vzorka_cela.describe(["Junction_Location"])
Junction_Location.collect()

[Row(summary=u'count', Junction_Location=u'214577'),
 Row(summary=u'mean', Junction_Location=u'2.4355219804545687'),
 Row(summary=u'stddev', Junction_Location=u'3.135052740159779'),
 Row(summary=u'min', Junction_Location=u'-1'),
 Row(summary=u'max', Junction_Location=u'8')]

In [31]:
Skidding_and_Overturning = vzorka_cela.describe(["Skidding_and_Overturning"])
Skidding_and_Overturning.collect()

[Row(summary=u'count', Skidding_and_Overturning=u'214577'),
 Row(summary=u'mean', Skidding_and_Overturning=u'0.22709796483313682'),
 Row(summary=u'stddev', Skidding_and_Overturning=u'0.7270188381920063'),
 Row(summary=u'min', Skidding_and_Overturning=u'-1'),
 Row(summary=u'max', Skidding_and_Overturning=u'5')]

In [32]:
Vehicle_Leaving_Carriageway = vzorka_cela.describe(["Vehicle_Leaving_Carriageway"])
Vehicle_Leaving_Carriageway.collect()

[Row(summary=u'count', Vehicle_Leaving_Carriageway=u'214577'),
 Row(summary=u'mean', Vehicle_Leaving_Carriageway=u'0.40655801879977815'),
 Row(summary=u'stddev', Vehicle_Leaving_Carriageway=u'1.4483770501859567'),
 Row(summary=u'min', Vehicle_Leaving_Carriageway=u'-1'),
 Row(summary=u'max', Vehicle_Leaving_Carriageway=u'8')]

In [33]:
Hit_Object_off_Carriageway = vzorka_cela.describe(["Hit_Object_off_Carriageway"])
Hit_Object_off_Carriageway.collect()

[Row(summary=u'count', Hit_Object_off_Carriageway=u'214577'),
 Row(summary=u'mean', Hit_Object_off_Carriageway=u'0.604925038564245'),
 Row(summary=u'stddev', Hit_Object_off_Carriageway=u'2.1597066902557605'),
 Row(summary=u'min', Hit_Object_off_Carriageway=u'-1'),
 Row(summary=u'max', Hit_Object_off_Carriageway=u'11')]

In [34]:
Casualty_Reference = vzorka_cela.describe(["Casualty_Reference"])
Casualty_Reference.collect()

[Row(summary=u'count', Casualty_Reference=u'214577'),
 Row(summary=u'mean', Casualty_Reference=u'1.573039048919502'),
 Row(summary=u'stddev', Casualty_Reference=u'2.024951480543275'),
 Row(summary=u'min', Casualty_Reference=u'1'),
 Row(summary=u'max', Casualty_Reference=u'91')]

In [35]:
Casualty_Severity = vzorka_cela.describe(["Casualty_Severity"])
Casualty_Severity.collect()

[Row(summary=u'count', Casualty_Severity=u'214577'),
 Row(summary=u'mean', Casualty_Severity=u'2.8802760780512355'),
 Row(summary=u'stddev', Casualty_Severity=u'0.353317973182534'),
 Row(summary=u'min', Casualty_Severity=u'1'),
 Row(summary=u'max', Casualty_Severity=u'3')]

In [36]:
cast_vzorky= vzorka_cela.drop("Location_Easting_OSGR",
					 "Location_Northing_OSGR",
					"Longitude",
					 "Latitude",
					 "Local_Authority_(District)",
					 "1st_Road_Class",
					 "1st_Road_Number",
					 "Road_Type",
					 "Speed_limit",
					 "2nd_Road_Class",
					 "2nd_Road_Number",
					"Pedestrian_Crossing-Human_Control",
					"Pedestrian_Crossing-Physical_Facilities",
					 "Special_Conditions_at_Site",
					 "Carriageway_Hazards",
					 "Urban_or_Rural_Area",
					 "Vehicle_Reference",
					 "Towing_and_Articulation",
					 "Vehicle_Manoeuvre",
					"Vehicle_Location-Restricted_Lane",
					 "Vehicle_Leaving_Carriageway",
					 "Was_Vehicle_Left_Hand_Drive?",
					 "Journey_Purpose_of_Driver",
					 "Propulsion_Code",
					 "Driver_IMD_Decile",
					 "Driver_Home_Area_Type",
					"Vehicle_Reference",
					 "Casualty_Reference",
					 "Police_Force",
                     "Day_of_Week",
                     "Light_Conditions",
                     "Weather_Conditions",
                     "Road_Surface_Conditions",
                     "Vehicle_Type",
                     "Hit_Object_in_Carriageway",
                      "1st_Point_of_Impact",
                     "Sex_of_Driver",
                     "Age_of_Driver",
                    "Age_Band_of_Driver",
                    "Engine_Capacity_(CC)",
                    "Age_of_Vehicle",
                    "Casualty_Class",
                    "Sex_of_Casualty",
                    "Age_of_Casualty",
                    "Age_Band_of_Casualty",
                    "Casualty_Severity",
                    "Pedestrian_Location",
                    "Pedestrian_Movement",
                    "Car_Passenger",
                    "Bus_or_Coach_Passenger",
                    "Pedestrian_Road_Maintenance_Worker",
                    "Casualty_Type",
                    "Casualty_Home_Area_Type")
names = cast_vzorky.schema.names
print(names)

['Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Junction_Detail', 'Junction_Control', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_off_Carriageway']


In [37]:
training_data, test_data = cast_vzorky.randomSplit([0.6, 0.4], seed=123)

In [38]:
training_data.count()

128497

In [42]:
#Modelovanie
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.feature import VectorAssembler

vector_data = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"],
        outputCol="features").transform(cast_vzorky)
training_data, test_data = vector_data.randomSplit([0.7, 0.3], seed=123)
 

In [43]:
vector_data.count()

214577

In [44]:
#Decision tree classifier
tree_classifier = DecisionTreeClassifier(featuresCol="features",labelCol="Accident_Severity",impurity="entropy",maxDepth=10, maxBins=100) 
 
tree_model = tree_classifier.fit(training_data)
 
predictions = tree_model.transform(test_data)

#print(tree_model.toDebugString)
test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)


Testing error: 0.0194


In [48]:
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.9806312657855184)


In [45]:
# Select example rows to display.
predictions.select("prediction", "Accident_Severity", "features").show(5)

+----------+-----------------+--------------------+
|prediction|Accident_Severity|            features|
+----------+-----------------+--------------------+
|       2.0|                1|[1.0,1.0,8.0,4.0,...|
|       2.0|                1|[1.0,2.0,0.0,-1.0...|
|       2.0|                1|[1.0,3.0,3.0,4.0,...|
|       2.0|                1|[2.0,3.0,3.0,4.0,...|
|       2.0|                1|(8,[0,1,3,4],[2.0...|
+----------+-----------------+--------------------+
only showing top 5 rows



In [46]:
print(tree_model.toDebugString)


DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4ac3bfcc8f43be3d12f2) of depth 10 with 767 nodes
  If (feature 1 <= 3.0)
   If (feature 4 <= 1.0)
    If (feature 3 <= 0.0)
     If (feature 1 <= 1.0)
      If (feature 0 <= 1.0)
       If (feature 6 <= 1.0)
        If (feature 7 <= 3.0)
         If (feature 5 <= -1.0)
          If (feature 7 <= 0.0)
           If (feature 6 <= 0.0)
            Predict: 2.0
           Else (feature 6 > 0.0)
            Predict: 2.0
          Else (feature 7 > 0.0)
           Predict: 2.0
         Else (feature 5 > -1.0)
          If (feature 7 <= 2.0)
           If (feature 6 <= 0.0)
            Predict: 2.0
           Else (feature 6 > 0.0)
            Predict: 2.0
          Else (feature 7 > 2.0)
           If (feature 6 <= 0.0)
            Predict: 2.0
           Else (feature 6 > 0.0)
            Predict: 2.0
        Else (feature 7 > 3.0)
         If (feature 7 <= 4.0)
          If (feature 3 <= -1.0)
           If (feature 6 <= 0.0)
    

In [47]:
y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           1       0.62      0.05      0.09      1276
           2       0.98      1.00      0.99     63261

   micro avg       0.98      0.98      0.98     64537
   macro avg       0.80      0.53      0.54     64537
weighted avg       0.97      0.98      0.97     64537



array([[   65,  1211],
       [   39, 63222]], dtype=int64)

In [32]:
#SVM - nechapem prečo hadže chybu že SVM robi iba binarnu klasifikaciu a naš cieľovy #atribut ma 3 triedy - nema lebo som to upravil a ma len 2 takže nechapem fakt ..

from pyspark.ml.classification import LinearSVC
vzorka_cela.withColumn("Accident_Severity", when(vzorka_cela["Accident_Severity"] == -1, 2).otherwise(vzorka_cela["Accident_Severity"]))
vector_data = VectorAssembler(inputCols=["Number_of_Vehicles", "Number_of_Casualties", "Junction_Detail", "Junction_Control", "Did_Police_Officer_Attend_Scene_of_Accident", "Junction_Location","Skidding_and_Overturning","Hit_Object_off_Carriageway"],
        outputCol="features").transform(vzorka_cela)
svm_classifier = LinearSVC(
        featuresCol="features",             
        labelCol="Accident_Severity")                  

svm_model = svm_classifier.fit(training_data)

predictions = svm_model.transform(test_data)

test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)



Py4JJavaError: An error occurred while calling o184.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 153.0 failed 1 times, most recent failure: Lost task 3.0 in stage 153.0 (TID 13043, localhost, executor driver): java.io.FileNotFoundException: C:\Users\FIlip\AppData\Local\Temp\blockmgr-4b3fb485-f64f-4b1d-94ab-bc56b0b66c9b\3b\temp_shuffle_be5a30cc-238c-4c19-b8a1-93fb42782f3c (The system cannot find the path specified)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:299)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:238)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:102)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:115)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:235)
	at org.apache.spark.shuffle.sort.BypassMergeSortShu

In [39]:
# bayes- haže chybu že nevie pracovať s negativnymi hodnotami - stale to for nemame ak #tak to možem spraviť aj ručne na tych par atributov čo nam ostalo to nebude problem... 

from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
 
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol="features", labelCol="Accident_Severity")
 
# train the model
model = nb.fit(training_data)
 
predictions = model.transform(test_data)
#predictions.show()
 
evaluator = MulticlassClassificationEvaluator(labelCol="Accident_Severity", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))



Py4JJavaError: An error occurred while calling o558.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 179.0 failed 1 times, most recent failure: Lost task 1.0 in stage 179.0 (TID 13086, localhost, executor driver): java.io.FileNotFoundException: C:\Users\FIlip\AppData\Local\Temp\blockmgr-4b3fb485-f64f-4b1d-94ab-bc56b0b66c9b\05\temp_shuffle_462463b0-2791-422e-9a38-fdd24acbcadb (The system cannot find the path specified)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:299)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:238)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:102)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:115)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:235)
	at org.apache.spark.shuffle.sort.BypassMergeSortShu

In [49]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="Accident_Severity", featuresCol="features",impurity="entropy", numTrees=10, maxBins=100)

# Train model.  This also runs the indexers.
model = rf.fit(training_data)

# Make predictions.
predictions = model.transform(test_data)

#print(model.toDebugString)`

test_error = predictions.filter(predictions["prediction"] != predictions["Accident_Severity"]).count() / float(test_data.count())
print "Testing error: {0:.4f}".format(test_error)

Testing error: 0.0196


In [50]:
#vyhodnotenie - RandomForest 
predictions = model.transform(test_data)

y_true = predictions.select(['Accident_Severity']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print classification_report(y_true, y_pred)
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           1       1.00      0.01      0.01      1276
           2       0.98      1.00      0.99     63261

   micro avg       0.98      0.98      0.98     64537
   macro avg       0.99      0.50      0.50     64537
weighted avg       0.98      0.98      0.97     64537



array([[    9,  1267],
       [    0, 63261]], dtype=int64)

In [51]:
#presnost RandomForest 
from sklearn.metrics import accuracy_score
print('Accuracy score: ', accuracy_score(y_true, y_pred))

('Accuracy score: ', 0.9803678510002014)


In [52]:
# Gradient boosted trees -vychadza Testing error: 1.0000
#Root Mean Squared Error (RMSE) on test data = 0.134178
#netušim či to je dobre … testing error 1 znamena že 100% chyba  ? :D 

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

gbt_reg = GBTRegressor(featuresCol="features", labelCol="Accident_Severity", maxIter=10, maxBins=100)
 
# Train model. 
model = gbt_reg.fit(training_data)
 
# Make predictions.
predictions = model.transform(test_data)
 
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Accident_Severity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


Test Error = 1


In [137]:
#vyhodnotenie - Gradient boosted trees
#predictions = model.transform(test_data)

#y_true = predictions.select(['Accident_Severity']).collect()
#y_pred = predictions.select(['prediction']).collect()

#from sklearn.metrics import classification_report, confusion_matrix
#print classification_report(y_true, y_pred)
#confusion_matrix(y_true, y_pred)

In [138]:
#from sklearn.metrics import accuracy_score
#print('Accuracy score: ', accuracy_score(y_true, y_pred))

In [53]:
from pyspark.ml.clustering import KMeans
#Trains a k-means model.
kmeans = KMeans().setK(3).setSeed(1234)
model = kmeans.fit(training_data)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(training_data)
print("Within Set Sum of Squared Errors = " + str(wssse))
print("------------------------------------------------")
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)    
    
print("------------------------------------------------")
#detekovanie anomalii
for center in centers:
    for point in center:
        if (point > 5 or -5 > point):
               print "anomalia: {0:.15f}".format(point)

Within Set Sum of Squared Errors = 3067387.30004
------------------------------------------------
Cluster Centers: 
[2.16279465 1.86458322 4.02222398 3.63316833 1.17691968 4.13475981
 0.14607569 0.35833626]
[ 2.43323572  2.22355621  0.01776476 -0.83660322  1.1295754   0.00875224
  0.34268749  0.96650399]
[50.06968641 67.59233449  0.11498258 -0.84320557  1.          0.15679443
  0.22648084  0.42160279]
------------------------------------------------
anomalia: 50.069686411149824
anomalia: 67.592334494773525
